In [1]:
import sys
sys.path.append("./Bspli/")
import os  
import faiss
import time
import numpy as np 
import index 
import torch
import pandas as pd

mnist = np.load("dataset/mnist-784-euclidean.npy")
print(f'mnist data shape: {mnist.shape}')
print(f'mnist dtype: {mnist.dtype}')

# mnist = np.random.uniform(low=0, high=255, size=(100000, 2))
# mnist = mnist.astype(np.float32)
# print(f'mnist dtype: {mnist.dtype}')
# print(f'mnist data shape: {mnist.shape}')


mnist data shape: (60000, 784)
mnist dtype: float32


In [2]:
%%time
# Brute Search
flat = faiss.IndexFlatL2(mnist.shape[1])
flat.add(mnist)

D, FLAT_I = flat.search(mnist[0].reshape(1, mnist.shape[1]), k=100) 
print(f'brute query: {FLAT_I}')

brute query: [[    0 32248  8728 18932 30483 24149 42338 52295 26251 50173 53634 24330
  54159 57528  1482 53428 18123 31379 52864 10536 29719 36087 30489 23947
  20034 52057 33825 21654 31008 55208 22477 44282 47968 54203 19825  1634
  27378 33909 15378 24708 34474 26413 16017 46824 46358  1516 34557 16832
  21629 29021 10740 24107  5688 52665  1864  5036 39031  1978 40546 22322
  52231 37284 24730  5970 21976 16945  9568 36697 25675 54189 11396 42555
  33445 52540 44263 18404 19186 24232 54184 25762 14736 33970  5210 59212
   8642 22569 15052  2933  6772 22963  6516   832 21244 21583 35838 59846
  21210 13502 52559 13862]]
CPU times: total: 125 ms
Wall time: 115 ms


In [3]:
mnist_tensor = torch.from_numpy(mnist)
print(f'mnist tensor shape: {mnist_tensor.shape}')

idx = index.Indexing(
    gl_size=10000, 
    ll_size=2000,
    g_epoch_num=3,
    l_epoch_num=10,
    g_hidden_size=10,
    l_hidden_size=10,
    g_block_range=4,
    l_block_range=4,
    random_partitioning=False
)
idx.train(mnist_tensor)

print(f"local model len:{len(idx._l_model)}")

mnist tensor shape: torch.Size([60000, 784])
torch.Size([1995, 785])
torch.Size([8906, 785])
torch.Size([4610, 785])
torch.Size([3738, 785])
torch.Size([7329, 785])
torch.Size([4915, 785])
torch.Size([8447, 785])
torch.Size([4660, 785])
torch.Size([2740, 785])
torch.Size([2620, 785])
torch.Size([382, 785])
torch.Size([9658, 785])
first stage partitioning finish
partitioning blocks : 12
training local model


d:\conda\envs\ml\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


training local model
training local model
training local model
training local model
training local model
training local model
training local model
training local model
training local model
training local model
training local model
trainging global model
global index train smaple count: 132
finish
local model len:12


d:\study\bspli\./Bspli\utils\Partitioning.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ma = torch.tensor(means)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


In [4]:
%%time

def recall(pred, true):
    x = np.isin(pred, true)
    return x.sum() / true.size

qp = torch.from_numpy(mnist[0])
# print(qp)
pred = idx.query(qp, k=100)
print(f"recall: {recall(pred, FLAT_I)}")

pred = pred.to(torch.int)
print(f"pred: {pred}")

print(f"isin same block: {idx.isin(qp)}")

recall: 0.0
pred: tensor([24435, 14223, 48533,  1201, 16530,  6111, 26066,  1315, 35764, 46965,
         5531, 29306, 59473,  6251, 29279, 33664, 28065, 39489, 12223, 22268,
        43754, 17244, 20751, 15246, 35038, 36579,  6367, 44458,  1997, 23562,
        29493, 11661,  5938, 19237, 15458,  9852,  2598, 25357,  6409, 54636,
        28485,  6341, 28760,  7238, 12633, 14173, 50765, 21785,  8223, 29192,
        35736, 12580, 19485,  6357, 53946, 11027, 16220, 23063,  5399, 41883,
        13336, 11492, 36623, 31474, 27958,  6071,  1349, 15516, 23971, 54689,
        48161, 42681,  6891, 25252, 51646, 19769, 53903, 17454, 36677,  5977,
        22453, 36599, 23809, 21825, 38297, 33003, 22767, 50036,  6768,  6017,
        32274,  8897,  6093, 23125, 48139, 24075, 25840, 19213,  8896, 28365],
       dtype=torch.int32)
isin same block: False
CPU times: total: 15.6 ms
Wall time: 13 ms


In [5]:
result = []

def benchmark_knn_query(data, index, size=1000, k=100):
    indices = np.random.choice(data.shape[0], size, replace=False)
    query_time = 0
    cur_recall = 0
    isin_count = 0

    # query
    for i in indices:
        q = torch.from_numpy(data[i])
        start = time.time()
        qk = index.query(q, k=100, g_block_range=8, l_block_range=4)
        query_time += (time.time() - start)
        D, FLAT_I = flat.search(data[i].reshape(1, data.shape[1]), k=k) 
        cur_recall += recall(qk, FLAT_I)
        isin_count += 1 if index.isin(q) else 0
        
    result.append((query_time/1000, cur_recall/1000, isin_count/1000))

In [6]:
%%script false
# recall(pred, FLAT_I)

benchmark_knn_query(mnist, idx, size=1000, k=100)

In [ ]:
%%script false
print(result)
print(idx.get_search_blocks_num())

[(0.0885417001247406, 0.829970000000003, 0.082)]
59
